### Advent of Code 2020
#### Day 8: Handheld Halting

A series of instructions consisting of jumps, accumulator increments and no-ops has an infinite loop, but changing one no-op to a jump or vice versa will allow it to run to completion.

1. What's the value of the accumulator immediately before the instructions begin to loop?
2. After fixing the wrong instruction, what's the value of the accumulator at the end of execution?

No surprises in the preamble, just the usual AOC utility functions and the threading macros.

In [1]:
#lang iracket/lang #:require racket
(require advent-of-code
         threading)

The instructions are in a text file that looks like
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
Since we need to keep track of which instructions to jump to, I've turned it into a hash table so each instruction is indexed starting at 0.

In [2]:
(define raw-instructions (~> (fetch-aoc-input (find-session) 2020 8) (string-split "\n")))

(define instruction-set
  (for/hash ([instruction (in-list raw-instructions)] [i (in-naturals)])
    (match-define (list op val) (string-split instruction))
    (values i (cons (string->symbol op) (string->number val)))))

(for/list ([i (in-range 10)])
  (cons i (hash-ref instruction-set i)))


'((0 acc . 49) (1 jmp . 274) (2 acc . 49) (3 acc . 49) (4 jmp . 476) (5 jmp . 409) (6 jmp . 269) (7 jmp . 1) (8 acc . -11) (9 acc . 5))

#### Part 1

Now I write a little interpreter using structural pattern matching and recursion to execute the code.
* If the program tried to execute an instruction on the line immediately after the last instruction, the program terminates normally. (This won't happen in part 1, but it's important for part 2.)
* We track the line numbers that have been visited in each step.  If a number comes up a second time, that means we're about to start looping, so we break execution here.
* `acc n` instructions increment the accumulator by `n` and go to the next line.
* `jmp n` instructions skip up or down `n` instructions.
* `nop n` instructions don't do anything besides go to the next line.

In [3]:
(define (execute code [acc 0] [line 0] [visited (set)])
  (match (hash-ref code line 'terminated)
    ['terminated (cons 'terminated acc)]
    [_
     #:when (set-member? visited line)
     (cons 'looping acc)]
    [(cons 'acc n) (execute code (+ acc n) (add1 line) (set-add visited line))]
    [(cons 'jmp n) (execute code acc (+ n line) (set-add visited line))]
    [(cons 'nop _) (execute code acc (add1 line) (set-add visited line))]))

(execute instruction-set)


'(looping . 1949)

#### Part 2

So far so good.  Now we're told that flipping exactly one `jmp` to a `nop` or vice versa will fix the code, so let's write a utility function to perform that flip, identify the potential candidates for the fix and get an idea of what our workload will be for this problem.

In [4]:
(define (flip-op op)
  (match op
    [(cons 'jmp n) (cons 'nop n)]
    [(cons 'nop n) (cons 'jmp n)]))

(define instruction-count (hash-count instruction-set))
(define flippable-bits
  (filter (λ (i) (member (car (hash-ref instruction-set i)) '(jmp nop))) (range instruction-count)))

(length flippable-bits)


305

There's only 305 cases to check, so just starting from the top and trying each possible swap in sequence should work fine, rather than trying to come up with some fancier backtracking algorithm.  `for/or` stops at the first non-falsy result, so we just have to wait for the first result that matches the `(cons 'terminated _)` pattern.

In [5]:
(for/or ([i (in-list flippable-bits)])
  (define flipped-instruction-set (hash-update instruction-set i flip-op))
  (match (execute flipped-instruction-set)
    [(cons 'looping _) #f]
    [(and success (cons 'terminated _)) success]))


'(terminated . 2092)